In [2]:
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from base import BaseDataLoader
import pandas as pd
import gzip
import numpy as np
import re

In [9]:
def buildGlove():
    words = []
    idx = 0
    word2idx = {}
    # vectors = bcolz.carray(np.zeros(1), rootdir=f'./data/glove_6B_50.dat', mode='w')
    vectors = []
    with open(f'./data/glove.6B.50d.txt', 'rb') as f:
        for l in f:
            line = l.decode().split()
            word = line[0]
            words.append(word)
            word2idx[word] = idx
            idx += 1
            vect = np.array(line[1:]).astype(np.float)
            vectors.append(vect)
    vectors = np.reshape(vectors, (400000, 50))
    glove = {w: vectors[word2idx[w]] for w in words}
    return  glove

In [3]:
def buildIndexes():
    words = []
    idx = 0
    word2idx = {}
    # vectors = bcolz.carray(np.zeros(1), rootdir=f'./data/glove_6B_50.dat', mode='w')
    vectors = []
    with open(f'./data/glove.6B.50d.txt', 'rb') as f:
        for l in f:
            line = l.decode().split()
            word = line[0]
            words.append(word)
            word2idx[word] = idx
            idx += 1
#             vect = np.array(line[1:]).astype(np.float)
#             vectors.append(vect)
#     vectors = np.reshape(vectors, (400000, 50))
#     glove = {w: vectors[word2idx[w]] for w in words}
    return  word2idx

In [33]:
def buildGloveMatrix():
    words = []
    idx = 0
    word2idx = {}
    # vectors = bcolz.carray(np.zeros(1), rootdir=f'./data/glove_6B_50.dat', mode='w')
    vectors = []
    with open(f'./data/glove.6B.50d.txt', 'rb') as f:
        for l in f:
            line = l.decode().split()
            word = line[0]
            words.append(word)
            word2idx[word] = idx
            idx += 1
            vect = np.array(line[1:]).astype(np.float)
            vectors.append(vect)
    vectors = np.reshape(vectors, (400000, 50))
    glove = {w: vectors[word2idx[w]] for w in words}
    
    matrix_len = len(glove)
    weights_matrix = np.zeros((matrix_len, 50))
    words_found = 0
    for i, word in enumerate(glove):
        try: 
            weights_matrix[i] = glove[word]
            words_found += 1
        except KeyError:
            weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))
            
    return  weights_matrix

In [41]:
vect = buildGloveMatrix()
vect[0]

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01])

In [22]:
class AmznDataLoader():
        def __init__(self):
            self.df = self.getDF('./data/reviews_Amazon_Instant_Video_5.json.gz')
    #         display(self.df)
    #         print(self.df.loc[0][['overall','reviewText']])
            self.df = self.df[['reviewText', 'overall']]
            self.df['reviewText'] = self.df['reviewText'].apply(lambda x : self.title_parsing(x))
            X = self.df['reviewText']
            y = self.df['overall']-1
            word2idx = buildIndexes()
            self.maxNumberWords = 0
            for x in self.df['reviewText']:
                length = len(x)
                if length > self.maxNumberWords:
                    self.maxNumberWords = length
            # self.maxNumberWords = sorted(len(x) for x in df.reviewText)[-1]
            index_matrix = np.zeros((X.shape[0], self.maxNumberWords))
            words_found = 0
            for j, row in enumerate(X):
                for i, word in enumerate(row):
                    try:
                        index_matrix[j,i] = word2idx[word]
                        words_found += 1
                    except KeyError:
                        index_matrix[j, i] = 400009                        
            X = index_matrix
            X_train, self.X_test, y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
            
        def title_parsing(self, title):
            title = re.sub('[^a-zA-Z]', ' ', str(title))
            title = title.lower()
            title = title.split()
            return title
#         def get_batches(self, X, y, batch_size, glove):
#             batch_x = []
#             batch_y = []
#             choices = np.random.choice(X.shape[0], batch_size)
#             for idx in choices:
#                 batch_x.append(X.iloc[idx])
#                 batch_y.append(y.iloc[idx])
                
                
#             numberWords = len(batch_x)
#             glove_tensor = np.zeros((batch_size, 50, self.maxNumberWords))
#             words_found = 0
#             for j, sentence in enumerate(batch_x):
#                 for i, word in enumerate(sentence):
#                     try:
#                         glove_tensor[j,:, i] = glove[word]
#                         words_found += 1
#                     except KeyError:
#                         glove_tensor[j,:,i] = np.random.normal(scale=0.6, size=(50, ))
                        
#             batch_x = glove_tensor
            
#             return batch_x, batch_y
        
        def parse(self, path):
            g = gzip.open(path, 'rb')
            for l in g:
                yield eval(l)
            
        def getDF(self, path):
            i = 0
            df = {}
            for d in self.parse(path):
                df[i] = d
                i += 1
            return pd.DataFrame.from_dict(df, orient='index')

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.autograd import Variable

In [38]:
# todo reset parameters for linear layer
class FirstModel(nn.Module):
    def __init__(self):
        super(FirstModel, self).__init__()
        glove_matrix = buildGloveMatrix()
        self.embedding , num_emb, emb_dim = self.create_emb_layer(glove_matrix)
        self.drop1 = nn.Dropout(p=0.5)
        self.conv1 = nn.Conv1d(in_channels=50, out_channels=200, kernel_size=4, padding=2)
        self.conv2 = nn.Conv1d(in_channels=50, out_channels=200, kernel_size=5, padding=2)
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.drop2 = nn.Dropout(p=0.15)
        self.hx = Variable(torch.randn(1, 256, 100))
        self.rnn = nn.GRU(input_size=400, hidden_size=100, num_layers=1, batch_first=True)  
        self.fc1 =nn.Linear(in_features=1534*100, out_features = 400)
        self.drop3 = nn.Dropout(p=0.1)
        self.fc2 = nn.Linear(in_features=400, out_features=10)
    def forward(self, x):
        x = self.embedding(x)
        x = self.drop1(x)
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x))
        x1 = self.maxpool(x1)
        x2 = self.maxpool(x2)
        x = torch.cat((x1,x2), 1)
        x = self.drop2(x)
        #x = pack_padded_sequence(x, lengths, batch_first=True)
        x = x.view(256, 1534, 400)
        output, hidden = self.rnn(x, self.hx)
#         for i in range(10):
#             self.hx = self.rnn(x[i], self.hx)
#             output.append(self.hx)
        x = output.contiguous().view(256,-1)
        x = F.relu(self.fc1(x))
        x = self.drop3(x)
        x = self.fc2(x)
        
        return x
    def create_emb_layer(self, weights_matrix, non_trainable=True):
        num_embeddings, embedding_dim = weights_matrix.shape
        emb_layer = nn.Embedding(num_embeddings, embedding_dim)
        emb_layer.load_state_dict({'weight': weights_matrix})
        if non_trainable:
            emb_layer.weight.requires_grad = False

        return emb_layer, num_embeddings, embedding_dim
    

In [13]:
import torch.optim as optim

model = FirstModel().float()    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

NameError: name 'create_layer' is not defined

In [ ]:
# to do implement accuracy
#
#todo validation loss
#todd early stoppning
#plot loss

data_loader = AmznDataLoader()
glove = buildGlove()
print_every = 10
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(100):
        # get the inputs,
        inputs, labels = data_loader.get_batches(data_loader.X_train, data_loader.y_train, batch_size=256, glove=glove)
        inputs = torch.from_numpy(inputs).float()
        labels = torch.LongTensor(labels)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize,
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward(retain_graph=True)
        optimizer.step()

        # print statistics,
        running_loss += loss.item()
        if i % print_every == print_every-1:    # print every 2000 batches,
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / print_every))
            running_loss = 0.0

In [15]:
from skorch import NeuralNetClassifier
import torch.optim as optim

In [39]:
net = NeuralNetClassifier(FirstModel, 
                          max_epochs=5, 
                          lr=0.001, 
                          criterion = nn.CrossEntropyLoss, 
                          optimizer=optim.SGD,
                          optimizer__param_groups=[('momentum', 0.9)],
                          batch_size = 256
                          )


In [40]:
data_loader = AmznDataLoader()
net.fit(data_loader.X_train, data_loader.y_train)

TypeError: 'int' object is not callable